In [1]:
!pwd

/workspace


In [2]:
import os
from pathlib import Path

print(os.getcwd())
# home = os.path.expanduser("~")
# home_path = Path(home)
# print(home_path)
project_path = Path("/workspace/mlx-week8")
# project_path = Path(os.path.expanduser("~") + "/workspace/mlx-week8")
print(project_path)
os.chdir(project_path)

/workspace
/workspace/mlx-week8


In [3]:
import torch

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [5]:
import pandas as pd
from importlib import reload

# Convert the data to pandas and filter them

# Save all of these once and reuse them 

In [6]:
import pickle


def load(filename):
    with open(filename, "rb") as f:
        return pickle.load(f)

In [7]:
train_url_id_to_doctext_mapping = load("train_url_id_to_doctext_mapping.pkl")
test_url_id_to_doctext_mapping = load("test_url_id_to_doctext_mapping.pkl")
train_query_id_to_query_mapping = load("train_query_id_to_query_mapping.pkl")
test_query_id_to_query_mapping = load("test_query_id_to_query_mapping.pkl")
train_query_id_to_wfa = load("train_query_id_to_wfa.pkl")
test_query_id_to_wfa = load("test_query_id_to_wfa.pkl")

# The plan

## Generating Batches of data

### High Level:
  - Follow a generate -> serialize strategy to generate all the embeddings in FAISS for passages, step by step.
  - Create a template for the forward pass method to include the extra dimension for TopK docs.

0. Look up generation[DONE]
1. Embedding generation in memory for DOCS[DONE]
The noted files saved on disk are:
```
137M    test_encoded_passage_attention_mask.pt
137M    test_encoded_passage_tokens.pt 
173M    test_passage_embeddings.pt              -> TESTING: Embeddings for MARCO MS test set
504K    test_query_id_to_query_mapping.pkl      -> TESTING: Query ID -> Query Text mapping
1.2M    test_query_id_to_wfa.pkl                -> TESTING: QueryID to well formed answer mapping
21M     test_url_id_to_doctext_mapping.pkl      -> TESTING: URL -> Passage Text mapping
1.8G    train_encoded_passage_attention_mask.pt 
1.8G    train_encoded_passage_tokens.pt
6.1M    train_query_id_to_query_mapping.pkl    -> TRAINING: Query ID -> Query Text mapping
13M     train_query_id_to_wfa.pkl              -> TRAINING: QueryID to well formed answer mapping 
254M    train_url_id_to_doctext_mapping.pkl   -> TRAINING: URL -> Passage Text mapping
1.8G    train_passage_embeddings.pt         -> TRAINING: Embeddings for MARCO MS train set
```
2. I have to figure out how to create a batch of size B * T tokens to then encoded which are padded. 
NOTE: I did not use a collator the last time around for preparing my data for the tiny stories task. But I might this time.

  -  Data -> Dataset. I need to retrieve the doc IDs only during training time for getting the embeddings which would be B size vector with teh doc id of each passage. Same for the query IDs which I would use to retrive the query embeddings. Here would be the outoput from the torch dataset `__getitem__` method.

```
{
    "query_id":...
    # NOTE: these are Top K passage ids for the query
    "passage_ids":[], # We don't need anything the well formed answer because its 1:1 mapped to query_id.
}
```

  - Pre Generate all the emebedings for FAISS[IN_PROGRESS]
    - Keep all the embeddings with faiss in memory should not be that big, but lets test this.
  - Create the dataset and the loader to do this[TODO]
  - We can load the embeddings for the top K passages per the query from the specific query in the `__getitem__` method of the dataset return those passage IDs. 
  - I'll need to save `*_query_id_to_query_mapping`, `*_url_id_to_doctext_mapping` and `*_query_id_to_wfa`
        giving me access to all three things i need to make the dataset.
  -


3. The forward pass over the dataloader should load all the passages 
    -  In the loop:
        - Lets do the tokenization and padding in the loop using the convenient huggingface interface,
        this allows for:
          ```
          decoder_inputs = tokenizer(target_texts, padding="longest", return_tensors="pt")
          encoder_inputs = tokenizer(source_texts, padding="longest", return_tensors="pt")
          outputs = model(input_ids=encoder_inputs['input_ids'],
                attention_mask=encoder_inputs['attention_mask'],
                decoder_input_ids=decoder_inputs['input_ids'],
                decoder_attention_mask=decoder_inputs['attention_mask'])
          ```
        - The query embeddings will need to be generated on the fly from the tunable model.I think we will use the Pipeline 
        - This will distribute the load of the task between the dataloader and the model's forward pass.

## Forward pass
NEXT: Feed the query and concatenated passages to the encoder of BART model.
- [TODO]: FOllow ardavan's diagram to get the loss propagating correctly. Whats the advantage of hugging face's pipeline?
```
for batch in data_loader:
  query_ids = batch['query_id']
  passage_ids = batch['passage_ids']
  with torch.no_grad():
    query_embeddings = get_query_embeddings(query_ids, query_bert_model)
    passage_embeddings = get_query_embeddings(query_ids, passage_bert_model)
```


NEXT: Get the embeddings for the query and 


In [8]:
from transformers import (
    RobertaModel,
    BartModel,
    RobertaTokenizer,
    BartTokenizer,
    RobertaTokenizerFast,
)

# Load RoBERTa
# roberta = RobertaModel.from_pretrained('roberta-base')
# roberta_tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

In [9]:
from transformers import AutoTokenizer, AutoModelForTokenClassification

# roberta_tokenizer = AutoTokenizer.from_pretrained("roberta-base")
roberta_tokenizer = RobertaTokenizerFast.from_pretrained("FacebookAI/roberta-base")

In [10]:
roberta_tokenizer.is_fast

True

In [11]:
import faiss

# norms = np.linalg.norm(embeddings, axis=1, keepdims=True)
# normalized_embeddings = embeddings / norms
dimension = 768

# Create FAISS index for inner product (cosine similarity)
index = faiss.IndexIDMap(faiss.IndexFlatL2(dimension))

In [12]:
# Now lets get all the embeddings
# from transformers import RobertaModel, BartModel, RobertaTokenizer, BartTokenizer

# Load RoBERTa
# roberta = RobertaModel.from_pretrained('roberta-base')
# roberta_tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

In [13]:
import numpy as np

len(train_url_id_to_doctext_mapping),
print(
    np.quantile(
        [len(v.split()) for v in train_url_id_to_doctext_mapping.values()],
        [0.05, 0.5, 0.95, 0.99],
    )
)
print(max([len(v.split()) for v in train_url_id_to_doctext_mapping.values()]))

[ 30.  53. 106. 129.]
230


In [14]:
tokens = roberta_tokenizer.tokenize("Hello, my dog is cute")
print(tokens)
tokens = roberta_tokenizer.encode("Hello, my dog is cute", return_tensors="pt")
print(tokens)
roberta_tokenizer.decode(tokens[0])

['Hello', ',', 'Ġmy', 'Ġdog', 'Ġis', 'Ġcute']
tensor([[    0, 31414,     6,   127,  2335,    16, 11962,     2]])


'<s>Hello, my dog is cute</s>'

# Super fast tokenizers

In [15]:
encoded = roberta_tokenizer("My name is john I work at MLX")
encoded, encoded.is_fast

({'input_ids': [0, 2387, 766, 16, 41906, 38, 173, 23, 10725, 1000, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]},
 True)

In [17]:
(encoded.tokens(), encoded.word_ids())

(['<s>',
  'My',
  'Ġname',
  'Ġis',
  'Ġjohn',
  'ĠI',
  'Ġwork',
  'Ġat',
  'ĠML',
  'X',
  '</s>'],
 [None, 0, 1, 2, 3, 4, 5, 6, 7, 7, None])

In [18]:
all_passages = list(train_url_id_to_doctext_mapping.values())
all_test_passages = list(test_url_id_to_doctext_mapping.values())
# all_tokens = roberta_tokenizer.batch_encode_plus(all_passages, return_tensors='pt', padding=True, truncation=True)

In [16]:
roberta_tokenizer.is_fast

True

In [19]:
from importlib import reload

In [20]:
from src.mlx_week8 import data_utils

## Tokenize the training dataset

In [21]:
reload(data_utils)


def tokenize_passages(passages):
    return roberta_tokenizer(
        passages, padding=True, truncation=False, return_tensors="pt"
    )

In [22]:
import torch

tokenized = True
if not tokenized:
    encoded_inputs = tokenize_passages(all_passages)
    torch.save(encoded_inputs["input_ids"], "train_encoded_passage_tokens.pt")
    torch.save(
        encoded_inputs["attention_mask"], "train_encoded_passage_attention_mask.pt"
    )
else:
    pass  # Load it

## Tokenize the test dataset

In [23]:
import torch

tokenized = True
if not tokenized:
    # encoded_inputs = tokenize_passages(all_passages)
    encoded_test_inputs = tokenize_passages(all_test_passages)
    torch.save(encoded_test_inputs["input_ids"], "test_encoded_passage_tokens.pt")
    torch.save(
        encoded_test_inputs["attention_mask"], "test_encoded_passage_attention_mask.pt"
    )
else:
    pass  # Load it

# Pre-generate the embeddings for the passages.

## Test passage embeddings

In [24]:
reload(data_utils)

if tokenized:
    encoded_test_inputs = {
        "input_ids": torch.load("test_encoded_passage_tokens.pt"),
        "attention_mask": torch.load("test_encoded_passage_attention_mask.pt"),
    }
generated_embeddings = True
if not generated_embeddings:
    test_embeddings = data_utils.generate_embeddings(
        {
            "input_ids": encoded_test_inputs["input_ids"],
            "attention_mask": encoded_test_inputs["attention_mask"],
        },
        batch_size=512, 
        device="cuda",
    )
    torch.save(test_embeddings, "test_passage_embeddings.pt")
    print(test_embeddings.shape)

In [25]:
tokenized

True

## Train Passage Embeddings

In [26]:
if tokenized:
    encoded_train_inputs = {
        "input_ids": torch.load("train_encoded_passage_tokens.pt", map_location=device),
        "attention_mask": torch.load("train_encoded_passage_attention_mask.pt", map_location=device),
    }
generated_embeddings = False

if not generated_embeddings:
    train_embeddings = data_utils.generate_embeddings(
        encoded_train_inputs, batch_size=1024, device="cuda"
    )
    torch.save(train_embeddings, "train_passage_embeddings.pt")
    print(train_embeddings.shape)
    

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
 33%|███▎      | 217/655 [16:57<34:40,  4.75s/it]

# To faiss

In [ ]:
import numpy

ids = np.array(range(len(train_url_id_to_doctext_mapping)), dtype="int64")

In [ ]:
# Add embeddings along with their IDs
index.add_with_ids(embeddings, ids)

In [ ]:
# Now that we have the tokens